# Week 2 Day 5 Challange

In [ ]:
import asyncio
from typing import Dict, Union, List
from pydantic import BaseModel, Field
from agents import Agent, Runner, WebSearchTool, function_tool, handoff

# --- 1. Modele Danych (Pydantic) ---

class ClarificationRequest(BaseModel):
    questions: List[str] = Field(description="Pytania do użytkownika w celu doprecyzowania researchu")
    reason: str = Field(description="Dlaczego te informacje są potrzebne")

class WebSearchItem(BaseModel):
    query: str = Field(description="Fraza do wyszukiwania")
    reason: str = Field(description="Dlaczego to wyszukiwanie jest ważne")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description="Lista zaplanowanych wyszukiwań")

class ReportData(BaseModel):
    short_summary: str = Field(description="Krótkie podsumowanie")
    markdown_report: str = Field(description="Pełny raport w formacie Markdown z sekcją 'Free Tools'")
    follow_up: List[str] = Field(description="Sugestie dalszych kroków")

# --- 2. Definicje Agentów Specjalistów ---

planner_agent = Agent(
    name="PlannerAgent",
    instructions="""Zaplanuj 5 precyzyjnych wyszukiwań. Jeśli zapytanie jest zbyt ogólne, 
    użyj modelu ClarificationRequest, aby zadać 3 pytania doprecyzowujące (np. o darmowe narzędzia, Python, multi-agent).""",
    model="gpt-4o-mini",
    output_type=Union[WebSearchPlan, ClarificationRequest]
)

writer_agent = Agent(
    name="WriterAgent",
    instructions="""Stwórz szczegółowy raport (min. 1000 słów). 
    MUSISZ uwzględnić sekcję 'Free Tools' dla rozwiązań Open-Source w Pythonie.""",
    model="gpt-4o-mini",
    output_type=ReportData
)

# --- 3. Narzędzia Końcowe ---

@function_tool
def task_completed(summary: str):
    """Wywoływane przez Managera na samym końcu procesu."""
    return f"🏁 Zadanie ukończone! Podsumowanie: {summary}"

# --- 4. Manager jako Agent (Orkiestrator) ---

research_manager = Agent(
    name="ResearchManager",
    instructions="""Jesteś dyrygentem researchu. Twoim zadaniem jest:
    1. Przekazać zapytanie do PlannerAgent (handoff).
    2. Jeśli Planner poprosi o doprecyzowanie, przekaż to użytkownikowi.
    3. Po zebraniu danych, przekaż je do WriterAgent.
    4. Na samym końcu użyj narzędzia 'task_completed'.""",
    handoffs=[planner_agent, writer_agent],
    tools=[task_completed],
    model="gpt-4o-mini"
)

# --- Przykład uruchomienia (Workflow) ---
async def main():
    query = "Latest AI Agent frameworks in 2025"
    print(f"🚀 Uruchamiam research dla: {query}")
    
    # W architekturze handoffs Runner zarządza przepływem między agentami
    result = await Runner.run(research_manager, query)
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())